<a href="https://colab.research.google.com/github/msrjdin/TransformerUseCases/blob/master/Text%20Classification/distilbert-base-uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
df.shape

(67730, 5)

In [5]:
df.head()

,asin,Rating5,title,body1,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [6]:
df.rename(columns = {'body1':'text', 'Rating5':'5star'}, inplace=True)

In [7]:
df.reset_index(drop= True, inplace=True)

In [8]:
df.head()

,asin,5star,title,text,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [9]:
from sklearn.model_selection import train_test_split
X= df['text'].copy()
y= df['5star']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [11]:
#Setting the model_name runs the model required
model_name = "distilbert-base-uncased"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [12]:
tr_enc = tokenizer(list(X_train.values), padding=True, truncation=True, max_length=100)
tst_enc = tokenizer(list(X_test.values),  padding=True, truncation=True,  max_length=100)

In [13]:
import torch

class buildingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = buildingDataset(tr_enc, list(y_train.values))
val_dataset = buildingDataset(tst_enc, list(y_test.values))

In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

trainer = Trainer(
    model=pt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics
)


In [16]:
trainer.train()

{'loss': 0.6833534240722656, 'learning_rate': 5e-06, 'epoch': 0.0033738191632928477, 'step': 10}
{'loss': 0.6813721656799316, 'learning_rate': 1e-05, 'epoch': 0.006747638326585695, 'step': 20}
{'loss': 0.6677643775939941, 'learning_rate': 1.5e-05, 'epoch': 0.010121457489878543, 'step': 30}
{'loss': 0.6438650131225586, 'learning_rate': 2e-05, 'epoch': 0.01349527665317139, 'step': 40}
{'loss': 0.5753498077392578, 'learning_rate': 2.5e-05, 'epoch': 0.016869095816464237, 'step': 50}
{'loss': 0.4925872802734375, 'learning_rate': 3e-05, 'epoch': 0.020242914979757085, 'step': 60}
{'loss': 0.5194408416748046, 'learning_rate': 3.5e-05, 'epoch': 0.023616734143049933, 'step': 70}
{'loss': 0.5309646606445313, 'learning_rate': 4e-05, 'epoch': 0.02699055330634278, 'step': 80}
{'loss': 0.5692943572998047, 'learning_rate': 4.5e-05, 'epoch': 0.030364372469635626, 'step': 90}
{'loss': 0.6033500671386719, 'learning_rate': 5e-05, 'epoch': 0.033738191632928474, 'step': 100}
{'loss': 0.3653606414794922, 'le

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 0.397576904296875, 'learning_rate': 4.648249828414551e-05, 'epoch': 0.1720647773279352, 'step': 510}
{'loss': 0.43111419677734375, 'learning_rate': 4.639670555936857e-05, 'epoch': 0.17543859649122806, 'step': 520}
{'loss': 0.40169677734375, 'learning_rate': 4.631091283459163e-05, 'epoch': 0.1788124156545209, 'step': 530}
{'loss': 0.407794189453125, 'learning_rate': 4.622512010981469e-05, 'epoch': 0.18218623481781376, 'step': 540}
{'loss': 0.352618408203125, 'learning_rate': 4.613932738503775e-05, 'epoch': 0.18556005398110662, 'step': 550}
{'loss': 0.457379150390625, 'learning_rate': 4.605353466026081e-05, 'epoch': 0.18893387314439947, 'step': 560}
{'loss': 0.44730224609375, 'learning_rate': 4.596774193548387e-05, 'epoch': 0.19230769230769232, 'step': 570}
{'loss': 0.4609100341796875, 'learning_rate': 4.588194921070693e-05, 'epoch': 0.19568151147098514, 'step': 580}
{'loss': 0.44100341796875, 'learning_rate': 4.5796156485929995e-05, 'epoch': 0.199055330634278, 'step': 590}
{'lo

{'loss': 0.33658447265625, 'learning_rate': 2.5377487989018532e-05, 'epoch': 1.0020242914979758, 'step': 2970}
{'loss': 0.32808837890625, 'learning_rate': 2.5291695264241594e-05, 'epoch': 1.0053981106612686, 'step': 2980}
{'loss': 0.3097412109375, 'learning_rate': 2.5205902539464653e-05, 'epoch': 1.0087719298245614, 'step': 2990}
{'loss': 0.36583251953125, 'learning_rate': 2.5120109814687715e-05, 'epoch': 1.0121457489878543, 'step': 3000}
{'loss': 0.3422607421875, 'learning_rate': 2.5034317089910774e-05, 'epoch': 1.015519568151147, 'step': 3010}
{'loss': 0.23385009765625, 'learning_rate': 2.494852436513384e-05, 'epoch': 1.01889338731444, 'step': 3020}
{'loss': 0.33408203125, 'learning_rate': 2.4862731640356898e-05, 'epoch': 1.0222672064777327, 'step': 3030}
{'loss': 0.34197998046875, 'learning_rate': 2.477693891557996e-05, 'epoch': 1.0256410256410255, 'step': 3040}
{'loss': 0.30076904296875, 'learning_rate': 2.4691146190803022e-05, 'epoch': 1.0290148448043186, 'step': 3050}
{'loss': 0.

TrainOutput(global_step=5928, training_loss=0.3616393136913799)

In [17]:
trainer.evaluate()


{'eval_loss': 0.40294435931839656, 'eval_accuracy': 0.8434962350509375, 'eval_f1': 0.86340206185567, 'eval_precision': 0.8404415454089312, 'eval_recall': 0.8876523582405935, 'epoch': 2.0, 'step': 5928}


{'epoch': 2.0,
 'eval_accuracy': 0.8434962350509375,
 'eval_f1': 0.86340206185567,
 'eval_loss': 0.40294435931839656,
 'eval_precision': 0.8404415454089312,
 'eval_recall': 0.8876523582405935}